In [2]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os.path as osp
import cv2

In [3]:
class MNISTDataset(Dataset):
    def __init__(self, root, transform = None, preload = False):
        self.images = None
        self.labels = None
        self.fileNames = []
        self.root = root
        self.transform = transform
        
        for i in range(10):
            for filename in glob.glob(osp.join(root, str(i), "*.png")):
                self.fileNames.append((filename, i))

        self.len = len(self.fileNames)
        
        if preload: self.__preload()
        
    def __preload(self):
        self.labels = []
        self.images = []
        for fn, label in self.fileNames:
            self.images.append(cv2.cvtColor(cv2.imread(fn), cv2.COLOR_BGR2GRAY))
            self.labels.append(label)

    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        label = None
        image = None
        if self.images is not None:
            image = self.images[index]
            label = self.labels[index]
        else:
            fn, label = self.fileNames[index]
            image = cv2.imread(fn)
        
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [4]:
trainset = MNISTDataset("./mnist_png/training", 
                        transform = transforms.ToTensor(), 
                        preload = True)

testset = MNISTDataset("./mnist_png/testing", 
                        transform = transforms.ToTensor(), 
                        preload = True)

In [5]:
trainsetLoader = DataLoader(trainset, batch_size = 64, shuffle = True, num_workers = 1)
testsetLoader = DataLoader(testset, batch_size = 1000, shuffle = False, num_workers = 1)
print(f"train set length = {len(trainset)}")
print(f"test set length = {len(testset)}")

train set length = 60000
test set length = 10000


In [6]:
import matplotlib.pyplot as plt
def imshow(img):
    plt.imshow(np.transpose(img.numpy(), (1, 2, 0)))

dataiter = iter(trainsetLoader)
image, label = dataiter.next()

imshow(torchvision.utils.make_grid(image))

In [7]:
useCuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda" if useCuda else "cpu")

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
        self.maxPool1 = nn.MaxPool2d(kernel_size = 2)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
        self.dropOut2 = nn.Dropout2d()
        self.maxPool2 = nn.MaxPool2d(kernel_size = 2)
        self.relu2 = nn.ReLU()
        
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = self.relu1(self.maxPool1(self.conv1(x)))
        x = self.relu2(self.maxPool2(self.dropOut2(self.conv2(x))))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        return F.log_softmax(self.fc2(x), dim = 1)

In [14]:
from time import time

def train(model, trainDataLoader, testDataLoader, epoch, logInterval = 100):
    model.train()
    iteration = 0
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    for e in range(epoch):
        start = time()
        for batchIndex, (data, label) in enumerate(trainDataLoader):
            data, label = data.to(device), label.to(device)
            output = model(data)
            loss = F.nll_loss(output, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if iteration % logInterval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    e, batchIndex * len(data), len(trainDataLoader.dataset),
                    100. * batchIndex / len(trainDataLoader), loss.item()))
            iteration += 1
        end = time()
        print('{:.2f}s'.format(end-start))

def test(model, testDataLoader):
    model.eval()
    testLoss = 0
    correct = 0
    with torch.no_grad():
        for data, target in testDataLoader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            testLoss = F.nll_loss(output, target, size_average=False).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    testLoss /= len(testDataLoader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        testLoss, correct, len(testDataLoader.dataset),
        100. * correct / len(testDataLoader.dataset)))

In [10]:
model = Net().to(device)
train(model, trainsetLoader, testsetLoader, 5)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.295003
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.301767
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.284818
Train Epoch: 0 [19200/60000 (32%)]	Loss: 2.254495
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.149849
Train Epoch: 0 [32000/60000 (53%)]	Loss: 1.841183
Train Epoch: 0 [38400/60000 (64%)]	Loss: 1.171233
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.969552
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.619123
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.604459
32.07s
Train Epoch: 1 [3968/60000 (7%)]	Loss: 0.400573
Train Epoch: 1 [10368/60000 (17%)]	Loss: 0.709443
Train Epoch: 1 [16768/60000 (28%)]	Loss: 0.514449
Train Epoch: 1 [23168/60000 (39%)]	Loss: 0.548188
Train Epoch: 1 [29568/60000 (49%)]	Loss: 0.472444
Train Epoch: 1 [35968/60000 (60%)]	Loss: 0.408589
Train Epoch: 1 [42368/60000 (71%)]	Loss: 0.379076
Train Epoch: 1 [48768/60000 (81%)]	Loss: 0.614471
Train Epoch: 1 [55168/60000 (92%)]	Loss: 0.558267
31.24s
Train Epoch: 2 [1536/60000 (3%)]	Loss: 0.301

In [15]:
test(model, testsetLoader)


Test set: Average loss: 0.0214, Accuracy: 9595/10000 (96%)

